In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
ax.set_title("Observation Likelihood vs. Num. Hidden States")
colormap = plt.colormaps['Reds']
color_offset = 0.2

for object_num in range(0, objects):
    color_value = (object_num / (objects - 1)) * (1 - color_offset) + color_offset
    color = colormap(color_value)

    ax.plot(hidden_states_set, scores_all[object_num], "-o", label=f"Object {object_num + 1}", color=color)
    ax.set_xticks(hidden_states_set)
    ax.set_xlabel("Number of Hidden States")
    ax.set_ylabel("Observation Likelihood")

ax.legend(loc="lower right")

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
ax.set_title("Change in Observation Likelihood vs. Num. Hidden States")
colormap = plt.colormaps['Reds']
color_offset = 0.2

for object_num in range(0, objects):
    diff = []
    diff = [scores_all[object_num][i] - scores_all[object_num][i-1] for i in range(1, len(hidden_states_set))]
    
    color_value = (object_num / (objects - 1)) * (1 - color_offset) + color_offset
    color = colormap(color_value)
    
    ax.plot(hidden_states_set[1:], diff, "-o", label=f"Object {object_num + 1}", color=color)
    ax.set_xticks(hidden_states_set)
    ax.set_xlabel("Number of Hidden States")
    ax.set_ylabel("Change in Observation Likelihood")

ax.legend(loc="upper right")

In [ ]:
 if object_num == 3:
                object_num_corr = 1
            elif object_num == 11:
                object_num_corr = 2
            elif object_num == 8:
                object_num_corr = 4
            elif object_num == 10:
                object_num_corr = 9
            else:
                object_num_corr = object_num
                
            if highest_likelihood_object == 3:
                highest_likelihood_object_corr = 1
            elif highest_likelihood_object == 11:
                highest_likelihood_object_corr = 2
            elif highest_likelihood_object == 8:
                highest_likelihood_object_corr = 4
            elif highest_likelihood_object == 10:
                highest_likelihood_object_corr = 9
            else:
                highest_likelihood_object_corr = highest_likelihood_object

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10,4))
#fig.subplots_adjust(hspace=0.2, top=0.8, wspace=0.4)
#fig.suptitle('Variations in Training Accuracy for Different Numbers of Hidden States')

axes.set_title("Training Accuracy for Varying Numbers of Hidden States", fontsize=10)
axes.plot(hidden_states_set, accuracies, "-o", color='red')
axes.set_xticks(hidden_states_set)
axes.set_xlabel("Number of Hidden States")
axes.set_ylabel("Accuracy")

#axes[1].set_title("Change in Training Accuracy for Varying Numbers of Hidden States", fontsize=10)
#diff = [accuracies[i] - accuracies[i-1] for i in range(1, len(hidden_states_set))]
#axes[1].plot(hidden_states_set[1:], diff, "-o", color='red')
#axes[1].set_xticks(hidden_states_set)
#axes[1].set_xlabel("Number of Hidden States")
#axes[1].set_ylabel("Change in Accuracy")

plt.show()

In [ ]:
#colormap = plt.colormaps['Reds']
#color_offset = 0.2

#for idx, start_state in enumerate(start_state_set):
#    color_value = (idx / (len(start_state_set) - 1)) * (1 - color_offset) + color_offset
#    color = colormap(color_value)
#    axes.plot(hidden_states_set, accuracies[start_state], ">", color=color, label=f'Start Seed: {start_state}')
#    axes.set_xticks(hidden_states_set)
#    axes.legend(loc='lower right', ncol=3)
#    axes.set_xlabel("Number of Hidden States")
#    axes.set_ylabel("Training Accuracy")

#for hidden_states in hidden_states_set:
#    plt.axvline(x=hidden_states, color='black', linestyle='dashed', linewidth=1, alpha=0.2)

In [ ]:
#parameter search (2)
channel = 0
scaling = True

#define train/test split
split = 0.7
train_nums, test_nums = random_selection(split)

#model parameters
model_name = 'GaussianHMM'
cov_type = 'tied'
start_state = 1

#initialize arrays
accuracies_train = {}
#------------------------------
accuracies_test = {}

iterations_run = 0

for hidden_states in hidden_states_set:
    accuracies_train[hidden_states] = np.empty((len(mfcc_features_set), len(window_factor_set)))
    accuracies_test[hidden_states] = np.empty((len(mfcc_features_set), len(window_factor_set)))
    mfcc_idx = 0
    for mfcc_features in mfcc_features_set:
        window_idx = 0
        for window_factor in window_factor_set:
            #extract data
            data, lengths = data_extraction(channel=channel,
                                            mfcc_features=mfcc_features,
                                            window_factor=window_factor,
                                            padding_factor=padding_factor,
                                            hop_factor=hop_factor,
                                            default_config=False,                                                        
                                            scaling=scaling) 

            train_data, test_data, train_lengths, test_lengths = split_data_fix(data=data,
                                                                                lengths=lengths,
                                                                                train_nums=train_nums,
                                                                                test_nums=test_nums)
            train_data_concatenated = concatenate_data(train_data)                                                             

            trained_models, conv_bool_set = model_training(hidden_states=hidden_states,
                                                           model_name=model_name,
                                                           cov_type=cov_type,
                                                           start_state=start_state,
                                                           train_data=train_data_concatenated,
                                                           train_lengths=train_lengths)
            
            real_labels_train, predicted_labels_train = model_testing(trained_models=trained_models,
                                                                      data=train_data)
            
            real_labels_test, predicted_labels_test = model_testing(trained_models=trained_models,
                                                                    data=test_data)
            
            accuracy_test = model_accuracy(real_labels_test, predicted_labels_test)
            accuracy_train = model_accuracy(real_labels_train, predicted_labels_train)

            accuracies_test[hidden_states][mfcc_idx][window_idx] = accuracy_test 
            accuracies_train[hidden_states][mfcc_idx][window_idx] = accuracy_train

            window_idx += 1

            iterations_run += 1
            update_and_display(f'Iterations Run: {iterations_run}')

        mfcc_idx += 1